# Asyncio:
This is a technique used in I/O intensive coding. API calls, network requests etc where concurrency is required.  

### 1_basic.py

A simple idea is to 
1. Write all the funtions with async: async def foo():
2. Call the function with await foo()
3. Run the async with asyncio.run(foo())


### 2_aiohttp.py
It is a basic idea for http call.
1. Create a (one) session with : async with aiohttp.ClientSession() as session:
2. Do the get call with async with session.get(url) as response:
Also focus on the fact that everything in async:
```{python, tidy=FALSE, eval=FALSE, highlight=FALSE }

    async def basic_aiohttp_session(): 
            async with aiohttp.ClientSession() as session: 
                async with session.get(url) as response: 
                    result = await response.json() 
                    ```

### 3_multiple_request.py

Similar to the previous one. Just be careful how to use the list
```{python, tidy=FALSE, eval=FALSE, highlight=FALSE }
async with aiohttp.ClientSession() as session:
        results = []
        for url in urls:
            async with session.get(url) as response:
                results.append(await response.json())
            ```

### 4_semaphore_multi.py
Use of semaphore limit the number of concurrent process.

1. Start the session and semaphore 
  ```{python, tidy=FALSE, eval=FALSE, highlight=FALSE }
    semaphore = asyncio.Semaphore(3)
    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch_with_semaphore(session, semaphore, url))
            for url in urls
        ]
        results = await asyncio.gather(*tasks)
  ```
2.  Notice the 'create_task(function(args)) and gather to start all the process.

3. Now the structure to use the semaphore 
```{python, tidy=FALSE, eval=FALSE, highlight=FALSE }
async def fetch_with_semaphore(session: aiohttp.ClientSession,
                              semaphore: asyncio.Semaphore,
                              url: str) -> Dict[str, Any]:
  
    async with semaphore:  
        try:
            async with session.get(url) as response:
                data = await response.json()

```

### 5_web_scrapper.py

Same as before, with fetch_with_retry

```{python, tidy=FALSE, eval=FALSE, highlight=FALSE }
async def fetch_with_retry(session: aiohttp.ClientSession,
                               semaphore: asyncio.Semaphore,
                               url: str,
                               max_retries: int = 3) -> Dict[str, Any]:
    async with semaphore:
        for attempt in range(max_retries):
            try:
                timeout = aiohttp.ClientTimeout(total=20)
                async with session.get(url, timeout=timeout) as response:
                    ......

semaphore = asyncio.Semaphore(2)  
    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch_with_retry(session, semaphore, url))
            for url in urls
        ]
        results = await asyncio.gather(*tasks,return_exceptions=True)

```



### 6_api_aggregator.py

api aggregator with fetch http.

### 7_performance.py

Compare the tasks with sync and async calls.

=== Performance Comparison ===
Performance: 5 URLs in 2.39s
Starting synchronous version (this will block)...
Synchronous version took 11.91s
